In [1]:
from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.cluster_graph import ClusterGraph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon
from package.user_proxy import UsersProxy
from package.itemset import Itemset

import pandas as pd
import networkx as nx
import math

In [3]:
# TOPICS = {
#     "Cluster": {
#         '1': [0.4139, 0.2764, 0.3097],
#         '2, 3, 5': [0.31980000000000003, 0.31626666666666664, 0.3639333333333333],
#         '6': [0.0213, 0.5398, 0.4389],
#         '0, 9': [0.11779999999999999, 0.47209999999999996, 0.4101],
#         '1, 3': [0.3639, 0.2671, 0.369],
#         '4': [0.4535, 0.5131, 0.0334],
#         '8': [0.0816, 0.0634, 0.855],
#         '3': [0.3139, 0.2578, 0.4283]

#     },
#     "Item": {
#         "iPhone": [0.7, 0.0, 0.3],
#         "AirPods": [0.9, 0.0, 0.1],
#         "Galaxy": [0.0, 0.8, 0.2],
#     }
# }

# PRICES = {
#     "iPhone": 260,
#     "AirPods": 60,
#     "Galaxy": 500,
# }
# RELATION = pd.DataFrame.from_dict({
#             "iPhone":{
#                 "AirPods":10,
#                 "Galaxy":-5
#             },
#             "AirPods":{
#                 "iPhone":1,
#                 "Galaxy":0,
#             },
#             "Galaxy":{
#                 "iPhone":-8,
#                 "AirPods":1,
#             }
#             })

TOPICS = {
    "Nodes": {
        '2543': [0.19015006121707884, 0.10805833054734416, 0.39984238221800494, 0.12739171749838238, 0.1745575085191897],

    },
    "Item": {
        "B000JX5JGI": [0.08011554159821069, 0.21047020618618262, 0.12759859152921385, 0.33202140065860875, 0.24979426002778404],
        "B000VYIUJS": [0.24266212485090108, 0.10409456331609068, 0.10142814295223984, 0.3170553745396183, 0.23475979434114994],
        "B000JX5JGI B000VYIUJS": [0.16138883322455588, 0.15728238475113665, 0.11451336724072685, 0.3245383875991135, 0.24227702718446698],
    }
}
PRICES = {
    "B000JX5JGI": 18.9,
    "B000VYIUJS": 12.31,
}
# edges = {('1', '2, 3, 5'): 2.3333333333333335,
#     ('1', '6'): 1.0,
#     ('2, 3, 5', '0, 9'): 2.0,
#     ('2, 3, 5', '1, 3'): 0.8333333333333333,
#     ('2, 3, 5', '4'): 0.75,
#     ('2, 3, 5', '8'): 0.75,
#     ('0, 9', '1'): 0.5,
#     ('1, 3', '2, 3, 5'): 2.3333333333333335,
#     ('1, 3', '4'): 0.5,
#     ('1, 3', '6'): 1.0,
#     ('4', '3'): 0.3333333333333333,
#     ('3', '4'): 0.5}

topic = TopicModel(3, TOPICS["Nodes"], TOPICS["Item"])
_itemset = ItemsetFlyweight(PRICES, topic)
_graph = ClusterGraph(cluster_topic=topic.getNodesTopic(), theta = 0.9, depth = 5)
_user_proxy = UsersProxy(_graph, _itemset, [])
# _graph.add_weighted_edges_from([(u, v, edges[u, v]) for u, v in edges.keys()])
for node in TOPICS["Nodes"].keys():
    _graph.add_node(node)
_graph._initAllNodes()

In [5]:
def getVPsByUserId(node, mainItemset=None, coupon=None):
    if bool(mainItemset) != bool(coupon):
        raise ValueError("if set mainItemset, coupon also should be set.")
    VP = dict()
    for obj in _itemset:
        try:
           ratio =  _user_proxy._VP_ratio(user_id=node, itemset=obj, mainItemset=mainItemset, coupon=coupon)
        except:
            ratio = -1

        VP[str(obj)] = ratio
    return VP


In [6]:
node = "2543"
_user_proxy._graph.nodes[node]["desired_set"] = _itemset["B000JX5JGI"]
# _user_proxy._graph.nodes[node]["adopted_set"] = _itemset[""]
# coupon = Coupon(accThreshold=_itemset["Galaxy"].price, 
#                         accItemset=_itemset["Galaxy"], 
#                         discount=175,
#                         disItemset=_itemset["iPhone"])

result = getVPsByUserId(node)
# result = getVPsByUserId(node, mainItemset=_itemset["AirPods"], coupon=coupon)
# _user_proxy._graph.nodes[node]["desired_set"] = None

In [7]:
result = sorted(result.items(), key= lambda item:item[1],reverse=True)
print("{}'s CP".format(node))
for k, v in result:
    print("{}: {}".format(k,v))
print()

result = sorted(result, key= lambda item:item[1]*_itemset[item[0]].price,reverse=True)
print("{}'s benfit".format(node))
for k, v in result:
    print("{}: {}".format(k,v*_itemset[k].price))
print()


2543's CP
B000VYIUJS: 0.01456661953158344
B000JX5JGI: 0.009253788317642312
B000JX5JGI B000VYIUJS: 0.005674650522864976

2543's benfit
B000VYIUJS: 0.17931508643379215
B000JX5JGI B000VYIUJS: 0.1771058428186159
B000JX5JGI: 0.1748965992034397



In [20]:
math.ceil(_user_proxy._min_discount("1", "Galaxy", "Galaxy iPhone"))

175

In [21]:
PRICES = {
    "iPhone": 260,
    "AirPods": 60,
    "Galaxy": 500,
}

In [22]:
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.itemset import ItemsetFlyweight
from package.user_proxy import UsersProxy
from package.coupon import Coupon

import math

TOPICS = {
    "Cluster": {
        '1': [0.4139, 0.2764, 0.3097],
        '2, 3, 5': [0.31980000000000003, 0.31626666666666664, 0.3639333333333333],
        '6': [0.0213, 0.5398, 0.4389],
        '0, 9': [0.11779999999999999, 0.47209999999999996, 0.4101],
        '1, 3': [0.3639, 0.2671, 0.369],
        '4': [0.4535, 0.5131, 0.0334],
        '8': [0.0816, 0.0634, 0.855],
        '3': [0.8, 0.2, 0.4283]

    },
    "Item": {
            '0': [0.7, 0.3],
            '1': [0.8, 0.2],
            '2': [0.5, 0.5],
            '0 1': [0, 1],
            '0 2': [0.4, 0.6],
            '1 2': [0.3, 0.7],
            '0 1 2': [0.6, 0.7]
        }
}
PRICES = {"0":200, "1":260, "2":60}
edges = {('1', '2, 3, 5'): 2.3333333333333335,
    ('1', '6'): 1.0,
    ('2, 3, 5', '0, 9'): 2.0,
    ('2, 3, 5', '1, 3'): 0.8333333333333333,
    ('2, 3, 5', '4'): 0.75,
    ('2, 3, 5', '8'): 0.75,
    ('0, 9', '1'): 0.5,
    ('1, 3', '2, 3, 5'): 2.3333333333333335,
    ('1, 3', '4'): 0.5,
    ('1, 3', '6'): 1.0,
    ('4', '3'): 0.3333333333333333,
    ('3', '4'): 0.5}

topic = TopicModel(3, TOPICS["Cluster"], TOPICS["Item"])
_itemset = ItemsetFlyweight(PRICES, topic)
_graph = SN_Graph(node_topic=TOPICS["Cluster"], located=False)
_user_proxy = UsersProxy(_graph, _itemset, [])
_graph.add_edges_from([(u, v) for u, v in edges.keys()])
_graph.initAttr()


In [23]:
node = "1"

result = _user_proxy.getVPsByUserId(node)
result = sorted(result.items(), key= lambda item:item[1],reverse=True)
print("{}'s CP".format(node))
for k, v in result:
    print("{}: {}".format(k,v))
print()

result = sorted(result, key= lambda item:item[1]*_itemset[item[0]].price,reverse=True)
print("{}'s benfit".format(node))
for k, v in result:
    print("{}: {}".format(k,v*_itemset[k].price))
print()


1's CP
2: 0.005752499999999999
0: 0.0018632499999999999
1: 0.0014861538461538462
0 2: 0.0012746153846153847
1 2: 0.00099265625
0 1 2: 0.0008496538461538461
0 1: 0.0006008695652173913

1's benfit
0 1 2: 0.44182
1: 0.3864
0: 0.37265
2: 0.34514999999999996
0 2: 0.33140000000000003
1 2: 0.31765
0 1: 0.2764

